<img src="SalesKen.png" alt="upGrad" align="Left" style="width: 200px;"/>

## SalesKen Data Science Test
<i><b>Author: Anish Mahapatra</i></b>

<i>Date: 01-Feb-2019</i>

Problem Statement - 1: Matching the misspelt cities.
There are two data files (.csv)

<b>Correct_cities.csv</b> : This file consists of a list of cities and they are spelt correctly. It has three columns "name" which is a city name; "country" where the city belongs to and "id" which is a unique id for each city.

<b>Misspelt_cities.csv</b> : This file consists of city names which are mispelt. Each city name has been misspelt by randomly replacing 20% of the characters by some random characters. It has two columns "misspelt_name" and "country".

Question : Write an algorithm or a group of algorithms to match the misspelt city name with the correct city name and return the list of unique ids for each misspelt city name.

For example : Correct Data -> (Bangalore, India, 101) and say for "Bangalore" the misspelt name is (Banhakorg, India). Then the matching algorithm should return 101 for this example.

<b>Approach 1:</b>

Steps to be followed to find and perform partial match:
1. Read the correct_cities.csv and misspelt_cities.csv file as a pandas dataframe
2. By country, iterate through the cities and compute the cosine similarity score
    - Substeps for cosine similarity: convert to lower case, find the partial match score
3. Set the threshold for the cosine similariity
4. If the cosine similarity score is greater than threshold, assign the id from the correct_cities onto the misspelt_cities file

<b>Next Steps:</b>
1. Look at the misspelt cities that still do not have an id
2. Use a different algorithm to perform partial text match on the files


In [15]:
# packages to be imported
import seaborn as sns
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

E:\SoftwareInstallations\Anaconda3\envs\Masters37\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
# Reading the csv files
correctCities = pd.read_csv("Correct_cities.csv")
misspeltCities = pd.read_csv("Misspelt_cities.csv")

## Correct cities

In [5]:
# Sense check of the data
correctCities.head()

,name,country,id
0,les Escaldes,Andorra,3040051
1,Andorra la Vella,Andorra,3041563
2,Umm al Qaywayn,United Arab Emirates,290594
3,Ras al-Khaimah,United Arab Emirates,291074
4,Khawr Fakkān,United Arab Emirates,291696


In [7]:
# Shape of the data
correctCities.shape

(23018, 3)

## Misspelt Cities

In [6]:
# Sense check of the data
misspeltCities.head()

,misspelt_name,country
0,Hfjdúszoposzló,Hungary
1,Otrajnyy,Russia
2,ian Isidre,Peru
3,Bordj Zemoufa,Algeria
4,ChulamViwta,United States


In [8]:
misspeltCities.shape

(23018, 2)

Awesome! 

Now we know that the shape of the data for both the data frames is the same and we should be able to match the correct files with the corrupted ones.

##### Let's convert the misspelt names and correct names to lowercase to ensure that the scores have inegrity

In [9]:
# Converting the column to lower case'
correctCities['name'].str.lower()

0            les escaldes
1        andorra la vella
2          umm al qaywayn
3          ras al-khaimah
4            khawr fakkān
               ...       
23013            bulawayo
23014             bindura
23015          beitbridge
23016             epworth
23017         chitungwiza
Name: name, Length: 23018, dtype: object

In [10]:
# Converting the column to lower case'
misspeltCities['misspelt_name'].str.lower()

0        hfjdúszoposzló
1              otrajnyy
2            ian isidre
3         bordj zemoufa
4           chulamviwta
              ...      
23013        zdigulegsk
23014            ygring
23015         moudougou
23016            bhīkxi
23017    belvfbere park
Name: misspelt_name, Length: 23018, dtype: object

Let's now feed this back to the original dataframe

In [13]:
# Feeding lower case back to original dataframe
correctCities['name'] = correctCities['name'].str.lower()
misspeltCities['misspelt_name'] = misspeltCities['misspelt_name'].str.lower()

### Fuzzywuzzy

In [17]:
# Sense check to generate the score between two strings
fuzz.ratio("SalesKen","Slaeeenek")

59

In [20]:
# Sense check to generate the partial score between two strings
fuzz.partial_ratio("SalesKen","Slaeeenek")

62

We shall be using the **fuzz.ratio** function as the order of the cities will remain the same 

In [26]:
# Function to check get the scores
def checker(wrong_options,correct_options):
    names_array=[]
    ratio_array=[]    
    for wrong_option in wrong_options:
        if wrong_option in correct_options:
            names_array.append(wrong_option)
            ratio_array.append('100')
        else:   
            x=process.extractOne(wrong_option,correct_options,scorer=fuzz.token_set_ratio)
            names_array.append(x[0])
            ratio_array.append(x[1])
    return names_array,ratio_array

Taking the columns to be matched and converting them to a list

In [36]:
# Converting to list
correctCitiesList = correctCities['name'].fillna('######').tolist()
misspeltCitiesList = misspeltCities['misspelt_name'].fillna('######').tolist()

In [37]:
# Using the function to match 
name_match,ratio_match=checker(misspeltCitiesList,correctCitiesList)

KeyboardInterrupt: 

In [ ]:
df1 = pd.DataFrame()
df1['old_names']=pd.Series(str2Match)
df1['correct_names']=pd.Series(name_match)
df1['correct_ratio']=pd.Series(ratio_match)
df1.to_excel('matched_names.xlsx', engine='xlsxwriter')